In [4]:
### 问题二
## 1.常规思路

import numpy as np
from pyqubo import Array, Constraint, Placeholder
import pandas as pd
from openjij import SQASampler

# 贷款利息收益率
R = 0.08

# 约束系数 M
M = 2

# matrix shape
N1 = 10
N2 = 100

# 读取附件 csv
ht = pd.read_csv('data_100.csv') 
hv = ht.values
h1 = []
t1 = []

# 建立 t, h array
for o in range(hv.shape[1]):
    if (o%2 == 0):
        t1.append(hv[:,o])
    else:
        h1.append(hv[:,o])

t = np.array([np.empty(N1)]*N2)
h = np.array([np.empty(N1)]*N2)

for k in range(N2):
    for l in range(N1):
        t[k][l] = t1[k][l]
        h[k][l] = h1[k][l]


# 建立 x, y, z Vector
x = Array.create('x', N1, 'BINARY')
y = Array.create('y', N1, 'BINARY')
z = Array.create('z', N1, 'BINARY')

## 定义约束条件和目标函数
# 约束条件 1, 2, 3：
# 在每一张卡中，选且只选 1 张信用评分卡
H1 = np.sum([(np.sum([x[i] for i in range(N1)]) - 1)**2])
H2 = np.sum([(np.sum([y[i] for i in range(N1)]) - 1)**2])
H3 = np.sum([(np.sum([z[i] for i in range(N1)]) - 1)**2])

# 计算 H0(Loss)
H0 = np.sum([(1.08 * (h[0, i] + h[1, j] + h[2, k])/3 * (t[0, i] * t[1, j] * t[2, k]) - 0.08 * (t[0, i] * t[1, j] * t[2, k])) * x[i] * y[j] * z[k] for i in range(N1) for j in range(N1) for k in range(N1)])

# 目标函数
H = H0 + Placeholder('M') * (Constraint(H1, label='H1') + Constraint(H2, label='H2') + Constraint(H3, label='H3'))
model = H.compile()

feed_dict = {'M': M}

# 求出 QUBO 矩阵
qubo, offset = model.to_qubo(feed_dict=feed_dict)


# Sample(使用 SQASampler 模拟退火)
sampler = SQASampler()

# # num_reads
R = 2000

sampleset = sampler.sample_qubo(qubo, num_reads=R)

# Save to ./xx/xxx_x_xxx.csv
save_path = './save_csv/ans_2_1_{}iter_1.csv'.format(R)

save = pd.DataFrame(sampleset)
save.to_csv(save_path)

# 输出结果
index_card = []

for i, row in save.iloc[[0],:].iterrows():
    for j in range(len(row)):
        if row[j] == 1:
            outrow = row.index[j]
            if len(outrow)==4:
                index_card.append(row.index[j].split('[')[1][0])

print('第一张信用评分卡选择第 {} 个阈值'.format(int(index_card[0]) + 1))
print('第二张信用评分卡选择第 {} 个阈值'.format(int(index_card[1]) + 1))
print('第三张信用评分卡选择第 {} 个阈值'.format(int(index_card[2]) + 1))

第一张信用评分卡选择第 8 个阈值
第二张信用评分卡选择第 1 个阈值
第三张信用评分卡选择第 2 个阈值


C:\Users\Administrator\AppData\Local\Temp\ipykernel_22144\243825364.py:85: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[j] == 1:
